In [1]:
# Raw Package
import numpy as np
import pandas as pd
import os
from actor_critic import *
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
#Data Source 
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import gym
from Stock_market_env import CustomDiscreteEnv

In [2]:
data = pd.DataFrame(yf.download(tickers='HDFCBANK.NS', period='5d', interval='15m'))
data = data.reset_index(drop = True)
env = CustomDiscreteEnv(data)

[*********************100%***********************]  1 of 1 completed


In [4]:
if __name__=='__main__':
    directory = 'D:\Reinforcement Learning\My codes\Custom gym'
    env_name = 'stock trading test'
    
    if (not os.path.exists(directory+'/'+'models/'+str(env_name)+'/')):
        os.makedirs(directory+'/'+'models/'+str(env_name)+'/')
    if (not os.path.exists(directory +'/'+ 'plots/'+str(env_name)+'/')):
        os.makedirs(directory+'/'+'plots/'+str(env_name)+'/')
    if (not os.path.exists(directory+'/'+'pickle/'+str(env_name)+'/')):
        os.makedirs(directory+'/'+'pickle/'+str(env_name)+'/')
      
    figure_file = directory+'/'+'plots/'+str(env_name)+'/'+env_name +'.png'
    pickle_file  = directory+'/'+'pickle/'+str(env_name)+'/'
    model_file  =directory+'/'+'models/'+str(env_name)+'/'
      
        
    N=20
    batch_size = 5
    n_epochs =5
    alpha = 0.0001
    n_steps, total_n_steps,learn_iters = 0, 0, 0
    n_episodes = 10
    score_history = []
    max_steps = 500
    prev_n_episodes = 0
    best_score = -np.inf
    input_dims = (1, env.observation_space.shape[0], env.observation_space.shape[1])
    
    train = True
    load_checkpoint = False
    
    agent = Agent(n_actions= 3,
                  lr = alpha, fc1_dims=256, fc2_dims=256,
                  input_dims= input_dims, pickle_file=pickle_file, chkpt_dir=model_file)
    
    if load_checkpoint:
        agent.load_models()
        score_history = pkl.load(agent.pklObj)
        prev_n_episodes = pkl.load(agent.pklObj)
        total_n_steps = pkl.load(agent.pklObj)
        learn_iters = pkl.load(agent.pklObj)
    
    
    for i in range(prev_n_episodes, n_episodes):
        observation = env.reset()
        done = False
        score = 0
        n_steps = 0
        while not done and n_steps<max_steps:
            action, prob, val = agent.choose_action(observation)
            
            observation_, reward, done, info = env.step(action)
            score += reward
            
            n_steps+=1
            total_n_steps += 1
            
            agent.remember(observation, action, prob, val, reward, done)
            if n_steps % N ==0:
                if train:
                    agent.learn()
                    learn_iters += 1
                    
                
            observation = observation_
            
        score_history.append(score)
        avg_score = np.mean(score_history[-20:])
        
        if train:  
            if score > best_score or (i == n_episodes-1):
                best_score = score
                agent.save_models()
                
                pkl.dump(score_history, agent.pklObj)
                pkl.dump(i, agent.pklObj)
                pkl.dump(total_n_steps, agent.pklObj)
                pkl.dump(learn_iters, agent.pklObj)
                
        
        
        print('Episode ',i , 'Score %.1f' %score, 'Best score %.1f ' %best_score ,'Avg score %.1f' %avg_score,
                'time_steps', total_n_steps, 'learning_steps', learn_iters)


KeyboardInterrupt

